In [7]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from backbone import ResNetBackbone, MobileNetBackbone

import numpy as np
import os
import argparse
import visdom
from tqdm import tqdm

from mtp import MTP, MTP_baseline, MTPLoss
import util

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
batch_size = 8
num_workers = 8
shuffle = False
is_diff = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = util.DataSet_proj('./dataset_chh/' + 'train_val', 'train_val')
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

parser = argparse.ArgumentParser()
parser.add_argument('--name',       required=True,  type=str,   help='experiment name. saved to ./exps/[name]')
parser.add_argument('--ep',         required=True,  type=str)



_StoreAction(option_strings=['--ep'], dest='ep', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

## Attention

In [3]:

args = parser.parse_args('--name 1130_mode1_152_att_1 --ep best'.split(' '))
exp_path, train_path, val_path, infer_path, ckpt_path = util.make_path(args)
# model = torch.load(ckpt_path + '/' + 'model.archi')
model = MTP(backbone=ResNetBackbone('resnet152'))
model.load_state_dict(torch.load(ckpt_path + '/' + 'weight_' + args.ep + '.pth')['state_dict'])

./exps/1130_mode1_152_att_1 already exsits.


<All keys matched successfully>

In [4]:
model = model.to(device)
model.eval()
transform_dac = transforms.Compose([transforms.ToPILImage(), transforms.Resize(500), transforms.ToTensor()])

loss_val_mean = []
dac = 0.
num_samp = 0
ade = 0.
fde = 0.
for raster, road, lane, agents, state, past, gt in tqdm(dataloader):
    raster, road, lane, agents, state, past, gt = util.NaN2Zero(raster), util.NaN2Zero(road), util.NaN2Zero(lane), util.NaN2Zero(agents), util.NaN2Zero(state), util.NaN2Zero(past),  util.NaN2Zero(gt)
    raster, road, lane, agents, state, past, gt = raster.to(device), road.to(device), lane.to(device), agents.to(device), state.to(device), past.to(device), gt.to(device)

    prediction = model(road, lane, agents, state, past)

    for road_, pred_, gt_ in zip(road, prediction, gt):
        road_ = util.restore_img(road_.cpu())
        road_ = transform_dac(road_).numpy()
        road_ = (road_ * 255).astype(np.uint8)

        dac_ = util.dac_metric(road_, pred_)
        dac += dac_

        gt_e = gt_.view(-1)
        pred_e = pred_[:-1]

        ade_ = ((gt_e - pred_e)**2).sum(dim=0) / len(gt_e)
        ade += ade_.item()

        fde_ = ((gt_e[-2:] - pred_e[-2:])**2).sum(dim=0) / 2
        fde += fde_.item()

        num_samp += 1

print('\n', 'dac : ', dac/num_samp)
print('\n', 'ade : ', ade/num_samp)
print('\n', 'fde : ', fde/num_samp)



100%|██████████| 392/392 [01:37<00:00,  4.01it/s]
 dac :  0.9649276749627735

 ade :  19.174236933420808

 fde :  70.30943596842414



## Difference

In [3]:
exp_path = 'exps/1201_mode1_mbnet_diff_1'
ckpt_path = exp_path + '/ckpt'
ep = 'best'

model = MTP_baseline(backbone=MobileNetBackbone('mobilenet_v2'), num_modes=1, is_diff=True)
model.load_state_dict(torch.load(ckpt_path + '/' + 'weight_' + ep + '.pth')['state_dict'])

dataset = util.DataSet_proj('./dataset_chh/' + 'train_val', 'train_val')
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

In [11]:
model = model.to(device)
model.eval()
transform_dac = transforms.Compose([transforms.ToPILImage(), transforms.Resize(500), transforms.ToTensor()])

loss_val_mean = []
dac = 0.
num_samp = 0
ade = 0.
fde = 0.
for raster, road, lane, agents, state, past, gt, diff in tqdm(dataloader):
    raster, road, lane, agents, state, past, gt, diff = util.NaN2Zero(raster), util.NaN2Zero(road), util.NaN2Zero(lane), util.NaN2Zero(agents), util.NaN2Zero(state), util.NaN2Zero(past),  util.NaN2Zero(gt), util.NaN2Zero(diff)
    raster, road, state, gt, diff = raster.to(device), road.to(device), state.to(device), gt.to(device), diff.to(device)

    prediction = model(raster, state)

    for road_, pred_, gt_ in zip(road, prediction, gt):
        road_ = util.restore_img(road_.cpu())
        road_ = transform_dac(road_).numpy()
        road_ = (road_ * 255).astype(np.uint8)

        ########## convert predicted displacement to coordinate #########
        tra_ = pred_[:-1]
        tra_ = torch.reshape(tra_, (-1,2))
        tra_[0] = tra_[0] + gt_[0]

        for i in range(len(tra_)-1):
            tra_[i+1] = tra_[i] + tra_[i+1]

        pred_ = torch.cat((gt_[0], tra_.reshape(-1), pred_[-1:]))
        ##################################################################

        gt_e = gt_.view(-1)
        pred_e = pred_[:-1]

        ade_ = ((gt_e - pred_e)**2).sum(dim=0) / len(gt_e)
        ade += ade_.item()

        fde_ = ((gt_e[-2:] - pred_e[-2:])**2).sum(dim=0) / 2
        fde += fde_.item()

        dac_ = util.dac_metric(road_, pred_)
        dac += dac_
        num_samp += 1

print('\n', 'dac : ', dac/num_samp)
print('\n', 'ade : ', ade/num_samp)
print('\n', 'fde : ', fde/num_samp)

100%|██████████| 392/392 [03:26<00:00,  1.90it/s]
 dac :  0.9241384811742178

 ade :  21.58204356461155

 fde :  79.69678150122152



In [8]:
tra_ = pred_[:-1]
tra_ = torch.reshape(tra_, (-1,2))
tra_[0] = tra_[0] + gt_[0]

for i in range(len(tra_)-1):
    tra_[i+1] = tra_[i] + tra_[i+1]

pred__ = torch.cat((gt_[0], tra_.reshape(-1), pred_[-1:]))

In [10]:
pred__

tensor([1.5136e-01, 2.1945e+00, 3.9808e-01, 6.6412e+00, 8.4048e-01, 1.3486e+01,
        1.4682e+00, 2.2763e+01, 2.3084e+00, 3.4366e+01, 3.4426e+00, 4.8137e+01,
        4.9762e+00, 6.3944e+01, 6.9345e+00, 8.1991e+01, 9.4172e+00, 1.0246e+02,
        1.2306e+01, 1.2553e+02, 1.5684e+01, 1.5115e+02, 1.9598e+01, 1.7934e+02,
        1.0000e+00], device='cuda:0', grad_fn=<CatBackward>)

## Baseline

In [9]:
# from mtp_orig import MTP, MTPLoss


exp_path = 'exps/1201_mode1_mbnet_1'
ckpt_path = exp_path + '/ckpt'
ep = 'best'

# model = torch.load(ckpt_path + '/' + 'model.archi')
# print(model)
model = MTP_baseline(backbone=MobileNetBackbone('mobilenet_v2'), num_modes=1, is_diff=False)
model.load_state_dict(torch.load(ckpt_path + '/' + 'weight_' + ep + '.pth')['state_dict'])

<All keys matched successfully>

In [10]:
model = model.to(device)
model.eval()
transform_dac = transforms.Compose([transforms.ToPILImage(), transforms.Resize(500), transforms.ToTensor()])

loss_val_mean = []
dac = 0.
num_samp = 0
ade = 0.
fde = 0.
for raster, road, lane, agents, state, past, gt, _ in tqdm(dataloader):
    raster, road, lane, agents, state, past, gt = util.NaN2Zero(raster), util.NaN2Zero(road), util.NaN2Zero(lane), util.NaN2Zero(agents), util.NaN2Zero(state), util.NaN2Zero(past),  util.NaN2Zero(gt)
    raster, road, state, gt = raster.to(device), road.to(device), state.to(device), gt.to(device)

    prediction = model(raster, state)

    for road_, pred_, gt_ in zip(road, prediction, gt):
        road_ = util.restore_img(road_.cpu())
        road_ = transform_dac(road_).numpy()
        road_ = (road_ * 255).astype(np.uint8)

        gt_e = gt_.view(-1)
        pred_e = pred_[:-1]

        ade_ = ((gt_e - pred_e)**2).sum(dim=0) / len(gt_e)
        ade += ade_.item()

        fde_ = ((gt_e[-2:] - pred_e[-2:])**2).sum(dim=0) / 2
        fde += fde_.item()

        dac_ = util.dac_metric(road_, pred_)
        dac += dac_
        num_samp += 1

print('\n', 'dac : ', dac/num_samp)
print('\n', 'ade : ', ade/num_samp)
print('\n', 'fde : ', fde/num_samp)

100%|██████████| 392/392 [03:28<00:00,  1.88it/s]
 dac :  0.943629015103169

 ade :  24.752807428227083

 fde :  82.45944582745899



In [33]:
fde_.item()

22.403121948242188